A few references:

* [ARVIZ API](https://python.arviz.org/en/latest/api/index.html)
* [PYMC API](https://www.pymc.io/projects/docs/en/stable/api.html)
* [xarray API](https://docs.xarray.dev/en/stable/api.html)
* [daft](https://docs.daft-pgm.org/en/latest/): &nbsp; [daft & complete pooling](https://www.tensorflow.org/probability/examples/Multilevel_Modeling_Primer#41_complete_pooling_model)
* [graphviz](https://graphviz.readthedocs.io/en/stable/api.html): &nbsp; [pymc.model_to_graphviz](https://www.pymc.io/projects/docs/en/stable/api/generated/pymc.model_to_graphviz.html), <a href="https://www.pymc.io/projects/docs/en/stable/_modules/pymc/model_graph.html#:~:text=VarName%22%2C%20str)%0A%0A%0Aclass-,ModelGraph,-%3A%0A%20%20%20%20def%20__init__(self"> pymc.model_graph.ModelGraph</a>

<br>

# Preliminaries

In [ ]:
!python --version

<br>

## Environment

In [ ]:
import os
import pathlib

<br>

Set path

In [ ]:
os.chdir(path=str(pathlib.Path(os.getcwd()).parent))

In [ ]:
os.getcwd()

<br>

## Libraries

In [ ]:
%matplotlib inline

In [ ]:
import logging
import collections

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

import numpy as np
import pandas as pd
import seaborn as sns

import arviz as az
import pymc as pm
import xarray as xr
import aesara.tensor as at
import graphviz

<br>

Versions

In [ ]:
az.__version__

In [ ]:
pm.__version__

In [ ]:
np.__version__

<br>

## Custom

In [ ]:
import src.graphics.settings
import src.graphics.sketch

<br>

Aesthetics

In [ ]:
settings = src.graphics.settings.Settings()

settings.layout()
settings.aesthetics()

<br>

For diagrams/figures

In [ ]:
sketch = src.graphics.sketch.Sketch()

In [ ]:
Labels = collections.namedtuple(typename='Labels', field_names=['title', 'xlabel', 'ylabel'])

<br>

## Settings

Seed

In [ ]:
RANDOM_SEED = 8924

<br>

Graphing

In [ ]:
az.style.use('arviz-darkgrid')

<br>

## Logging

In [ ]:
logging.basicConfig(level=logging.INFO, 
                    format='\n%(message)s\n%(asctime)s.%(msecs)03d\n', 
                    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)

<br>
<br>

# Data

## Dwelling Level

In [ ]:
try:
    data = pd.read_csv(filepath_or_buffer=pm.get_data('srrs2.dat'))
except FileNotFoundError as err:
    raise Exception(err.strerror)

In [ ]:
data.columns

In [ ]:
data.rename(mapper=str.strip, axis='columns', inplace=True)

In [ ]:
data.head()

<br>

Structuring; concatenating the `pure state` & `pure county` codes

* [FIPS States](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code)
* [FIPS Counties](https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county)


In [ ]:
data.loc[:, 'fips'] = data.stfips.astype(str).str.zfill(2) + data.cntyfips.astype(str).str.zfill(3)
data.head()

<br>

## County Level

In [ ]:
try:
    counties = pd.read_csv(filepath_or_buffer=pm.get_data('cty.dat'))
except FileNotFoundError as err:
    raise Exception(err.strerror)

In [ ]:
counties.loc[:, 'fips'] = counties.stfips.astype(str).str.zfill(2) + counties.ctfips.astype(str).str.zfill(3)

<br>

## Excerpting & Merging

Excerpt: Focus on Minnesota, MN, dwellings.

In [ ]:
excerpt = data.loc[data['state'] == 'MN', :]
excerpt.head()

<br>

Merge dwelling & county level [uranium] data.

In [ ]:
excerpt = excerpt.merge(counties[['fips', 'Uppm']], how='left', on='fips')
excerpt.columns

In [ ]:
excerpt.head()

<br>

## Duplicates

In [ ]:
logger.info(f'# of instances: {excerpt.shape}')
logger.info(f'# of unique instances: {excerpt.drop_duplicates().shape}')
logger.info(f"# of unique codes: {excerpt['idnum'].unique().shape}")

<br>

Hence

In [ ]:
excerpt.drop_duplicates(inplace=True)
excerpt['idnum'].unique().shape

<br>
<br>

# Explore

In [ ]:
excerpt.loc[:, 'ln_radon'] = np.log(excerpt['activity'] + 0.1)

In [ ]:
ax = sketch.figure(width=2.9, height=2.5)
ax.hist(excerpt.ln_radon, bins=25)
sketch.annotation(ax, Labels(title='', xlabel='bins', ylabel='frequency'))

<br>
<br>

# Model

## Simple

<br>

Snippets:

>  ```python
len(coords.get('Floors'))

> ```python 
with complete:
    logger.info(at.shape(epsilon).eval())
    
>```python
epsilon.eval()

<br>

Add more notes:

* A simple linear regression model: an intercept, a gradient whereby the accompanying independent variable is *dwelling level*
* The depedent variable is *ln(radon)*
* Is the *intercept* implicit below?

<br>

<br>

### Model

* `logger.info(level.get_value())`

In [ ]:
coords = {'LevelName': ['Basement', 'Ground']}

In [ ]:
with pm.Model(coords=coords) as complete:
    
    
    # the values of the <floor> field
    levelcode = pm.Data(name='levelcode', value=excerpt.floor.values, dims='N', mutable=True)
    logger.info(levelcode.get_value().shape)
    logger.info(levelcode.type())
    
    
    # <initial> probably has two elements - the object <Dwelling> has two elements, therefore two random values from a normal distribution
    initial = pm.Normal(name='initial', mu=0.0, sigma=10.0, dims='LevelName')
    
    
    # shape(mu) === shape(floor)
    mu = initial[levelcode]
    
    
    # model
    # pm.Exponential(name=, lam=)
    sigma = pm.Exponential('sigma', 1.0)
    y = pm.Normal('y', mu=mu, sigma=sigma, observed=excerpt['ln_radon'].values, dims='N')
    

<br>
<br>

Is this the correct seeding method/approach?

In [ ]:
complete.initial_point(seed=RANDOM_SEED)

In [ ]:
initial.eval()

<br>

Illustration of model

In [ ]:
pm.model_to_graphviz(complete)

In [ ]:
diagram = pm.model_graph.ModelGraph(model=complete).make_graph()
diagram.node_attr.update(shape='circle')
diagram.save(os.path.join(os.getcwd(), 'notebooks', 'simple.gv'))
graphviz.render(engine='dot', format='pdf', filepath=os.path.join(os.getcwd(), 'notebooks', 'simple.gv'));

In [ ]:
type(diagram)

<br>

### Prior Predictive Samples

An inspection of [prior predictive samples](https://www.pymc.io/projects/docs/en/stable/api/generated/pymc.sample_prior_predictive.html#pymc.sample_prior_predictive)

* `inferences.keys()`

* `inferences.get('prior').keys()`


In [ ]:
with complete:
    inferences = pm.sample_prior_predictive()
    

In [ ]:
inferences

<br>
<br>

**Alternative Graph**

* For the object's keys: 

>```python
prior.keys()

* Graphing:<br>[seaborn.boxplot](https://seaborn.pydata.org/generated/seaborn.boxplot.html)<br>[maxplotlib.axes.Axes.boxplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.boxplot.html#matplotlib.axes.Axes.boxplot)




The data

In [ ]:
prior = inferences.get('prior')

levelname = prior.get('LevelName').values

indices = np.asarray((np.where(levelname == 'Basement'))).squeeze()
basement = pd.DataFrame(data = {'initial': prior.get('initial').values[0, :, indices], 'level': indices, 'type': prior.get('LevelName').values[indices]})

indices = np.asarray((np.where(levelname == 'Ground'))).squeeze()
ground = pd.DataFrame(data = {'initial': prior.get('initial').values[0, :, indices], 'level': indices, 'type': prior.get('LevelName').values[indices]})

readings = pd.concat([basement, ground], axis = 0, ignore_index=True)

logger.info(readings.shape)
logger.info(readings.head())


<br>

An alternative graph

In [ ]:
ax = sketch.figure(width=2.1, height=2.9)
sns.boxplot(data=readings, x='type', y='initial', notch=True, color='k', showcaps=False, ax=ax)
sketch.annotation(ax, Labels(title='', xlabel='level', ylabel='mean(ln(radon))'))

<br>

### Modelling

References:
* [pymc.sample](https://www.pymc.io/projects/docs/en/stable/api/generated/pymc.sample.html)

In [ ]:
with complete:
    
    # starting of with the default sample settings
    trace = pm.sample(draws=1000, cores=None, tune=1000)

<br>

Hence

In [ ]:
logger.info(trace.keys())

<br>

`trace` will be thecore inference object, combining the `trace` & `inferences` objects $\longrightarrow$

In [ ]:
trace.extend(inferences)

<br>

### Metrics, Measures, Traces

In [ ]:
az.summary(trace)

<br>

Trace

In [ ]:
with complete:
    az.plot_trace(data=trace, figsize=(4.35, 2.95))

<br>

Cf.

In [ ]:
pm.plot_posterior(data=trace, var_names=['initial', 'sigma'], 
                 figsize=(7.65, 2.3), grid=(1, 3), point_estimate='median', textsize=9)

<br>
<br>

### Posterior Predictive Samples

In [ ]:
with complete:
    ppc = pm.sample_posterior_predictive(trace)

<br>

Hence, the resulting inferences

In [ ]:
logger.info(ppc.keys())

<br>

Append the latest inference data to the core inference object, i.e., `trace`

In [ ]:
trace.extend(ppc)
trace

<br>

### High Density Interval

A helper function
* [About `arviz.hdi(.)`](https://arviz-devs.github.io/arviz/api/generated/arviz.hdi.html)

* [About `xarray.DataArray`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html#xarray.DataArray)

* [xarray.DataArray.groupy](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.groupby.html#xarray.DataArray.groupby)

* cf. `trace.constant_data['level']` & `trace.constant_data['N']`

<br>

<br>

Posterior Predictive Check

In [ ]:
pm.plot_ppc(data=trace, figsize=(2.9, 2.4), num_pp_samples=125, random_seed=RANDOM_SEED, textsize=10)

In [ ]:
az.plot_ppc(data=trace, figsize=(2.9, 2.4), num_pp_samples=125, random_seed=RANDOM_SEED, textsize=10)

<br>

The following is incorrect, try a manual approach.

In [ ]:
PPC = az.hdi(trace.posterior_predictive['y'].groupby(group=trace.constant_data['levelcode']).var(), 
       input_core_dims=[['chain', 'draw']])

<br>

### Structuring


Co$\ddot{o}$rdinates & Labels

* [xarray API](https://docs.xarray.dev/en/stable/api.html)

* [xarray.Dataset.assign_coords](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.assign_coords.html#xarray.Dataset.assign_coords)



In [ ]:
labels = trace.posterior['LevelName'][trace.constant_data['levelcode']]
trace.observed_data = trace.observed_data.assign_coords(LevelName=labels)

<br>

Preview

In [ ]:
trace

<br>

### Graphs

In [ ]:
ax = sketch.figure(width=3.3, height=2.7)
sns.scatterplot(y=trace.observed_data['y'], x=trace.observed_data['LevelName'], ax=ax)
sketch.annotation(ax, Labels(title='', xlabel='level', ylabel='mean(ln(radon))'))

In [ ]:
ax = sketch.figure(width=3.3, height=2.7)
sns.scatterplot(y=trace.observed_data['y'], x=trace.observed_data['LevelName'], ax=ax)
az.plot_hdi([0,1], hdi_data=PPC, fill_kwargs={'alpha': 0.25, 'label': 'Exp. distrib. of Radon levels'},
    ax=ax)
sketch.annotation(ax, Labels(title='', xlabel='level', ylabel='mean(ln(radon))'))

In [ ]:
averages = trace.posterior.mean(dim=('chain', 'draw'))
averages